# Mikania Classification - Optimized Workflow

## Two-Phase Workflow

| Phase | Purpose | Run When |
|-------|---------|----------|
| **Phase 1** | Load raw data → Process → Export to GEE Asset | Once (first time only) |
| **Phase 2** | Load processed asset → Classify → Export results | Every time |

After Phase 1 exports complete, set `PHASE_1_COMPLETE = True` and run only Phase 2.

In [ ]:
import ee
import geemap

# =============================================================================
# WORKFLOW CONTROL
# =============================================================================
PHASE_1_COMPLETE = True  # Set True after processed mosaic is exported to GEE Asset

# Your GEE project and asset paths
PROJECT_ID = 'quantum-bonus-434714-t2'  # Replace with your project
PROCESSED_ASSET_PATH = 'projects/quantum-bonus-434714-t2/assets/GrasslandAreaProcessed'  # Where to save/load
UNIFIED_ASSET_PATH = 'projects/quantum-bonus-434714-t2/assets/AirbusMosaic_6band'  # Where to save/load

# Initialize
ee.Authenticate()  # Only needed onc
ee.Initialize(project=PROJECT_ID)
print('Earth Engine initialized!')

Earth Engine initialized!


---
# Phase 1: Data Processing Pipeline
**Skip this entire section after `PHASE_1_COMPLETE = True`**

In [ ]:
# =============================================================================
# VALIDATION: Check All Assets Exist and Load Correctly
# =============================================================================
print("Validating all image assets...")
# Complete asset list with paths
ALL_ASSETS = {
    # A tiles
    'A1R1C1': 'projects/daisyoneill/assets/A1R1C1',
    'A1R2C1': 'projects/daisyoneill/assets/A1R2C1',
    # B tiles
    'B_NED_R1C1': 'projects/daisyoneill/assets/B_NED_R1C1',
    'B_RGB_R1C1': 'projects/daisyoneill/assets/B_RGB_R1C1',
    'BR1C2': 'projects/daisyoneill/assets/BR1C2',
    'BR2C1': 'projects/daisyoneill/assets/BR2C1',
    'BR2C2': 'projects/daisyoneill/assets/BR2C2',
    # C tiles
    'CR1C1': 'projects/daisyoneill/assets/CR1C1',
    'CR1C2': 'projects/daisyoneill/assets/CR1C2',
    'CR2C2': 'projects/daisyoneill/assets/CR2C2',
    # D tiles
    'D2R1C1': 'projects/daisyoneill/assets/D2R1C1',
    'D2R1C2': 'projects/daisyoneill/assets/D2R1C2',
    'D2R2C1': 'projects/daisyoneill/assets/D2R2C1',
    'D2R2C2': 'projects/daisyoneill/assets/D2R2C2',
    # E tiles
    'ER1C1': 'projects/daisyoneill/assets/ER1C1',
    'ER1C2': 'projects/daisyoneill/assets/ER1C2',
    # F tiles
    'FR1C1': 'projects/daisyoneill/assets/FR1C1',
    'FR1C2': 'projects/daisyoneill/assets/FR1C2',
    # G tiles
    'GR1C1': 'projects/daisyoneill/assets/GR1C1',
    'GR1C2': 'projects/daisyoneill/assets/GR1C2',
    # H tiles
    'H_NED_R1C1': 'projects/daisyoneill/assets/H_NED_R1C1',
    'H_RGB_R1C1': 'projects/daisyoneill/assets/H_RGB_R1C1',
    'HR1C2': 'projects/daisyoneill/assets/HR1C2',
    'HR2C1': 'projects/daisyoneill/assets/HR2C1',
    'HR2C2': 'projects/daisyoneill/assets/HR2C2',
    # I tiles
    'I_NED_R1C1': 'projects/daisyoneill/assets/I_NED_R1C1',
    'I_RGB_R1C1': 'projects/daisyoneill/assets/I_RGB_R1C1',
    'IR1C2': 'projects/daisyoneill/assets/IR1C2',
    'IR2C1': 'projects/daisyoneill/assets/IR2C1',
    'IR2C2': 'projects/daisyoneill/assets/IR2C2',
    # J tiles
    'J_NED_R1C1': 'projects/daisyoneill/assets/J_NED_R1C1',
    'J_RGB_R1C1': 'projects/daisyoneill/assets/J_RGB_R1C1',
    'JR1C2': 'projects/daisyoneill/assets/JR1C2',
    'JR2C1': 'projects/daisyoneill/assets/JR2C1',
    'JR2C2': 'projects/daisyoneill/assets/JR2C2',
    # K tiles
    'K1R1C1': 'projects/daisyoneill/assets/K1R1C1',
    'K1R1C2': 'projects/daisyoneill/assets/K1R1C2',
    'K1R2C1': 'projects/daisyoneill/assets/K1R2C1',
    'K1R2C2': 'projects/daisyoneill/assets/K1R2C2',
    # L1 tiles
    'L1_NED_R1C1': 'projects/daisyoneill/assets/L1_NED_R1C1',
    'L1_RGB_R1C1': 'projects/daisyoneill/assets/L1_RGB_R1C1',
    'L1R1C2': 'projects/daisyoneill/assets/L1R1C2',
    'L1_NED_R3C1': 'projects/daisyoneill/assets/L1_NED_R3C1',
    'L1_RGB_R3C1': 'projects/daisyoneill/assets/L1_RGB_R3C1',
    'L1R3C2': 'projects/daisyoneill/assets/L1R3C2',
    'L1R4C1': 'projects/daisyoneill/assets/L1R4C1',
    'L1R4C2': 'projects/daisyoneill/assets/L1R4C2',
    # L3 tiles
    'L3_NED_R1C1': 'projects/daisyoneill/assets/L3_NED_R1C1',
    'L3_RGB_R1C1': 'projects/daisyoneill/assets/L3_RGB_R1C1',
    'L3R1C2': 'projects/daisyoneill/assets/L3R1C2',
    'L3R2C1': 'projects/daisyoneill/assets/L3R2C1',
    'L3R2C2': 'projects/daisyoneill/assets/L3R2C2',
    # M tiles
    'M2R1C1': 'projects/daisyoneill/assets/M2R1C1',
    'M2R1C2': 'projects/daisyoneill/assets/M2R1C2',
    # N tiles
    'N2R1C1': 'projects/daisyoneill/assets/N2R1C1',
    'N4R1C1': 'projects/daisyoneill/assets/N4R1C1',
    # O tiles (different user)
    'O_NED_R1C1': 'users/DaisyCrystalONeill/O_NED_R1C1',
    'O_RGB_R1C1': 'users/DaisyCrystalONeill/O_RGB_R1C1',
    'OR1C2': 'users/DaisyCrystalONeill/OR1C2',
    'O_NED_R2C1': 'users/DaisyCrystalONeill/O_NED_R2C1',
    'O_RGB_R2C1': 'users/DaisyCrystalONeill/O_RGB_R2C1',
    'OR2C2': 'users/DaisyCrystalONeill/OR2C2',
    # P tiles
    'P_NED_R1C1': 'projects/daisyoneill/assets/P_NED_R1C1',
    'P_RGB_R1C1': 'projects/daisyoneill/assets/P_RGB_R1C1',
    'PR1C2': 'projects/daisyoneill/assets/PR1C2',
    'PR2C1': 'projects/daisyoneill/assets/PR2C1',
    'PR2C2': 'projects/daisyoneill/assets/PR2C2',
}
# Validate each asset
passed = 0
failed = 0
failed_assets = []
for name, path in ALL_ASSETS.items():
    try:
        img = ee.Image(path)
        # Quick check: get band names (forces load)
        bands = img.bandNames().size().getInfo()
        print(f"  ✓ {name}: {bands} bands")
        passed += 1
    except Exception as e:
        print(f"  ✗ {name}: FAILED - {str(e)[:50]}")
        failed += 1
        failed_assets.append(name)
# Summary
print("\n" + "="*50)
print(f"VALIDATION SUMMARY: {passed}/{len(ALL_ASSETS)} assets loaded")
if failed > 0:
    print(f"⚠️  FAILED ASSETS ({failed}):")
    for name in failed_assets:
        print(f"    - {name}")
else:
    print("✅ All assets loaded successfully!")
print("="*50)

Validating all image assets...
  ✓ A1R1C1: 6 bands
  ✓ A1R2C1: 6 bands
  ✓ B_NED_R1C1: 3 bands
  ✓ B_RGB_R1C1: 3 bands
  ✓ BR1C2: 6 bands
  ✓ BR2C1: 6 bands
  ✓ BR2C2: 6 bands
  ✓ CR1C1: 6 bands
  ✓ CR1C2: 6 bands
  ✓ CR2C2: 6 bands
  ✓ D2R1C1: 6 bands
  ✓ D2R1C2: 6 bands
  ✓ D2R2C1: 6 bands
  ✓ D2R2C2: 6 bands
  ✓ ER1C1: 6 bands
  ✓ ER1C2: 6 bands
  ✓ FR1C1: 6 bands
  ✓ FR1C2: 6 bands
  ✓ GR1C1: 6 bands
  ✓ GR1C2: 6 bands
  ✓ H_NED_R1C1: 3 bands
  ✓ H_RGB_R1C1: 3 bands
  ✓ HR1C2: 6 bands
  ✓ HR2C1: 6 bands
  ✓ HR2C2: 6 bands
  ✓ I_NED_R1C1: 3 bands
  ✓ I_RGB_R1C1: 3 bands
  ✓ IR1C2: 6 bands
  ✓ IR2C1: 6 bands
  ✓ IR2C2: 6 bands
  ✓ J_NED_R1C1: 3 bands
  ✓ J_RGB_R1C1: 3 bands
  ✓ JR1C2: 6 bands
  ✓ JR2C1: 6 bands
  ✓ JR2C2: 6 bands
  ✓ K1R1C1: 6 bands
  ✓ K1R1C2: 6 bands
  ✓ K1R2C1: 6 bands
  ✓ K1R2C2: 6 bands
  ✓ L1_NED_R1C1: 3 bands
  ✓ L1_RGB_R1C1: 3 bands
  ✓ L1R1C2: 6 bands
  ✓ L1_NED_R3C1: 3 bands
  ✓ L1_RGB_R3C1: 3 bands
  ✓ L1R3C2: 6 bands
  ✓ L1R4C1: 6 bands
  ✓ L1R4C2: 6 band

In [ ]:
# =============================================================================
# 1) Asset Loading
# =============================================================================
if not PHASE_1_COMPLETE:
    print('Loading raw assets...')

    # Asset definitions - all tiles organized by region
    ASSETS = {
        # RGB+NED pairs (need stacking)
        'B': {'rgb': 'B_RGB_R1C1', 'ned': 'B_NED_R1C1', 'tiles': ['BR1C2', 'BR2C1']},
        'H': {'rgb': 'H_RGB_R1C1', 'ned': 'H_NED_R1C1', 'tiles': ['HR1C2', 'HR2C1', 'HR2C2']},
        'I': {'rgb': 'I_RGB_R1C1', 'ned': 'I_NED_R1C1', 'tiles': ['IR1C2', 'IR2C1', 'IR2C2']},
        'J': {'rgb': 'J_RGB_R1C1', 'ned': 'J_NED_R1C1', 'tiles': ['JR1C2', 'JR2C1', 'JR2C2']},
        'L1_R1': {'rgb': 'L1_RGB_R1C1', 'ned': 'L1_NED_R1C1', 'tiles': ['L1R1C2']},
        'L1_R3': {'rgb': 'L1_RGB_R3C1', 'ned': 'L1_NED_R3C1', 'tiles': ['L1R3C2', 'L1R4C1', 'L1R4C2']},
        'L3': {'rgb': 'L3_RGB_R1C1', 'ned': 'L3_NED_R1C1', 'tiles': ['L3R1C2', 'L3R2C1', 'L3R2C2']},
        'P': {'rgb': 'P_RGB_R1C1', 'ned': 'P_NED_R1C1', 'tiles': ['PR1C2', 'PR2C1', 'PR2C2']},
        # Simple tiles (already 6-band)
        'A': {'tiles': ['A1R1C1', 'A1R2C1']},
        'C': {'tiles': ['CR1C1', 'CR1C2']},
        'D': {'tiles': ['D2R1C1', 'D2R1C2', 'D2R2C1', 'D2R2C2']},
        'E': {'tiles': ['ER1C1', 'ER1C2']},
        'F': {'tiles': ['FR1C1', 'FR1C2']},
        'G': {'tiles': ['GR1C1', 'GR1C2']},
        'K': {'tiles': ['K1R1C1', 'K1R1C2', 'K1R2C1', 'K1R2C2']},
        'M': {'tiles': ['M2R1C1', 'M2R1C2']},
        'N2': {'tiles': ['N2R1C1']},
        'N4': {'tiles': ['N4R1C1']},
    }

    # O tiles from different user
    O_ASSETS = {
        'rgb': 'users/DaisyCrystalONeill/O_RGB_R1C1',
        'ned': 'users/DaisyCrystalONeill/O_NED_R1C1',
        'rgb2': 'users/DaisyCrystalONeill/O_RGB_R2C1',
        'ned2': 'users/DaisyCrystalONeill/O_NED_R2C1',
        'tiles': ['users/DaisyCrystalONeill/OR1C2', 'users/DaisyCrystalONeill/OR2C2']
    }

    print('Asset definitions loaded!')
else:
    print('SKIPPING Phase 1: PHASE_1_COMPLETE = True')

SKIPPING Phase 1: PHASE_1_COMPLETE = True


In [ ]:
# =============================================================================
# VISUALIZATION: Tile Footprints After Loading
# =============================================================================
if not PHASE_1_COMPLETE:
  import geemap
  print("Building tile footprint visualization...")
  Map = geemap.Map()
  # Color palette for different tile groups
  group_colors = {
      'A': '#FF6B6B', 'B': '#4ECDC4', 'C': '#45B7D1', 'D': '#96CEB4',
      'E': '#FFEAA7', 'F': '#DDA0DD', 'G': '#98D8C8', 'H': '#F7DC6F',
      'I': '#BB8FCE', 'J': '#85C1E9', 'K': '#F1948A', 'L': '#82E0AA',
      'M': '#F8C471', 'N': '#AED6F1', 'O': '#D7BDE2', 'P': '#A3E4D7'
  }
  # Group tiles by letter prefix
  tile_groups = {}
  for name, path in ALL_ASSETS.items():
      # Get first letter as group
      group = name[0] if name[0].isalpha() else name[:2]
      if group not in tile_groups:
          tile_groups[group] = []
      tile_groups[group].append((name, path))
  # Add each group as a layer
  all_footprints = []
  for group, tiles in tile_groups.items():
      color = group_colors.get(group, '#888888')

      group_features = []
      for name, path in tiles:
          try:
              img = ee.Image(path)
              footprint = img.geometry()
              all_footprints.append(footprint)
              group_features.append(ee.Feature(footprint, {'name': name}))
          except:
              pass

      if group_features:
          fc = ee.FeatureCollection(group_features)
          styled = fc.style(fillColor=color + '40', color=color, width=2)
          Map.addLayer(styled, {}, f'Group {group} ({len(group_features)} tiles)', shown=True)
  # Add combined footprint boundary
  if all_footprints:
      combined = ee.Geometry.MultiPolygon(all_footprints).dissolve()
      Map.addLayer(
          ee.FeatureCollection([ee.Feature(combined)]).style(
              fillColor='00000000', color='FF0000', width=3
          ),
          {},
          'Combined Footprint'
      )
      Map.centerObject(combined, zoom=9)

  Map.addLayerControl()
  print(f"Loaded {len(all_footprints)} tile footprints")
  print("Legend: Colored = tile groups | Red = combined | Green = Cloud | Yellow = Management")
  Map

In [ ]:
# =============================================================================
# 2) Function Definitions
# =============================================================================
if not PHASE_1_COMPLETE:
    BASE_PATH = 'projects/daisyoneill/assets/'

    def load_image(path):
        """Load image, use full path if provided, else prepend base."""
        if path.startswith('users/') or path.startswith('projects/'):
            return ee.Image(path)
        return ee.Image(BASE_PATH + path)

    def stack_rgb_ned(rgb_path, ned_path):
        """Stack RGB and NED into 6-band image with renamed bands."""
        rgb = load_image(rgb_path)
        ned = load_image(ned_path)
        return ee.Image.cat([rgb, ned]).select(
            ['b1','b2','b3','b1_1','b2_1','b3_1']
        ).rename(['b1','b2','b3','b4','b5','b6'])

    def lookup(source_hist, target_hist):
        """Create histogram lookup table."""
        src_vals = source_hist.slice(1, 0, 1).project([0])
        src_counts = source_hist.slice(1, 1, 2).project([0])
        src_counts = src_counts.divide(src_counts.get([-1]))
        tgt_vals = target_hist.slice(1, 0, 1).project([0])
        tgt_counts = target_hist.slice(1, 1, 2).project([0])
        tgt_counts = tgt_counts.divide(tgt_counts.get([-1]))
        lut = src_counts.toList().map(lambda n: tgt_vals.get(tgt_counts.gte(n).argmax()))
        return {'x': src_vals.toList(), 'y': lut}

    def histogram_match(src_img, tgt_img, src_geom, tgt_geom):
        """Match source histogram to target for all 6 bands."""
        args = lambda g: {
            'reducer': ee.Reducer.autoHistogram(maxBuckets=4096, cumulative=True),
            'geometry': g,
            'scale': 3,  # 10x coarser = 100x fewer pixels, statistically equivalent
            'bestEffort': True
        }
        src = src_img.reduceRegion(**args(src_geom))
        tgt = tgt_img.reduceRegion(**args(tgt_geom))
        return ee.Image.cat([
            src_img.select([b]).interpolate(**lookup(src.getArray(b), tgt.getArray(b)))
            for b in ['b1','b2','b3','b4','b5','b6']
        ])
    def add_indices(img):
        """Add NDVI, SAVI, NDWI, EVI."""
        ndvi = img.normalizedDifference(['b4','b1']).rename('nd')
        savi = img.expression('((N-R)/(N+R+0.5))*1.5', {'N':img.select('b4'),'R':img.select('b1')}).rename('sa')
        ndwi = img.expression('(G-N)/(G+N)', {'G':img.select('b2'),'N':img.select('b4')}).rename('wi')
        evi = img.expression('((N-R)/((N+(6*R)-(7.5*B))+1))*2.5',
                             {'N':img.select('b4'),'R':img.select('b1'),'B':img.select('b3')}).rename('ev')
        return ee.Image.cat([img, ndvi, savi, ndwi, evi])

    def add_texture(img, size=11):
        """Add GLCM texture metrics."""
        texture = img.toInt32().glcmTexture(size)
        bands = [f'{b}_{m}' for b in ['b1','b2','b3','b4','b5','b6','nd','sa','wi','ev']
                 for m in ['var','ent','contrast']]
    print('Phsse 1 Helper functions defined!')

In [ ]:
# =============================================================================
# 4) Build Tile-level Mosaics
# =============================================================================
if not PHASE_1_COMPLETE:
    print('Building tile mosaics...')

    mosaics = {}
    tile_collections = {}

    for name, cfg in ASSETS.items():
        images = []
        # Stack RGB+NED if present
        if 'rgb' in cfg:
            images.append(stack_rgb_ned(cfg['rgb'], cfg['ned']))
        # Add other tiles
        for t in cfg.get('tiles', []):
            images.append(load_image(t))

        tile_collections[name] = ee.ImageCollection.fromImages(images)

    # Handle O tiles separately (different user path)
    O_images = [
        stack_rgb_ned(O_ASSETS['rgb'], O_ASSETS['ned']),
        stack_rgb_ned(O_ASSETS['rgb2'], O_ASSETS['ned2']),
        *[ee.Image(t) for t in O_ASSETS['tiles']]
    ]
    tile_collections['O'] = ee.ImageCollection.fromImages(O_images)

    # Get projection from first tile
    default_proj = tile_collections['P'].first().select('b1').projection()

    # Create mosaics
    for name, coll in tile_collections.items():
        mosaics[name] = coll.mosaic().setDefaultProjection(default_proj).set(
            'system:footprint', coll.geometry()
        )
    # Reference mosaic for histogram matching
    E_mosaic = mosaics['E']
    E_geom = tile_collections['E'].geometry()

    print(f'Built {len(mosaics)} mosaics!')

In [ ]:
# =============================================================================
# 5) Verify Tile-level Mosaics
# =============================================================================
if not PHASE_1_COMPLETE:
  # Test BEFORE histogram matching
  print("Testing mosaics (BEFORE histogram matching)...")
  for name in ['E', 'A', 'B']:
      if name in mosaics:
          img = mosaics[name]
          try:
              val = img.reduceRegion(
                  ee.Reducer.first(),
                  img.geometry().centroid(),
                  scale=10
              ).get('b1').getInfo()
              print(f"  {name}: b1 = {val}")
          except Exception as e:
              print(f"  {name}: ERROR - {e}")
      else:
          print(f"  {name}: NOT IN mosaics dict")

In [ ]:
# =============================================================================
# 6) Apply Histogram Matching
# =============================================================================
if not PHASE_1_COMPLETE:
    print('Applying histogram matching (using E as reference)...')

    matched = {}
    for name, mosaic in mosaics.items():
        if name == 'E':
            matched[name] = mosaic  # E is reference
        else:
            src_geom = tile_collections[name].geometry()
            matched[name] = histogram_match(mosaic, E_mosaic, src_geom, E_geom)

    print('Histogram matching complete!')

In [ ]:
# =============================================================================
# 7) Verify Histogram Matching
# =============================================================================
if not PHASE_1_COMPLETE:
  # Quick test: Do the matched images have data?
  print("Testing matched images...")
  for name in ['E', 'A', 'B']:  # Test a few
      if name in matched:
          img = matched[name]
          try:
              val = img.reduceRegion(
                  ee.Reducer.first(),
                  img.geometry().centroid(),
                  scale=10
              ).get('b1').getInfo()
              print(f"  {name}: b1 = {val}")
          except:
              print(f"  {name}: ERROR")

In [ ]:
# =============================================================================
# 8) Create Mosaic
# =============================================================================
if not PHASE_1_COMPLETE:
    print('Creating final processed mosaic...')

    order = ['O','P','L1_R1','L1_R3','J','H','G','I','K','A','B','C','D','E','F','L3','M','N2','N4']
    ordered_images = [matched[n] for n in order if n in matched]
    print(f'Number of matched images: {len(ordered_images)}')

    # Build mosaic footprint from tile_collections
    mosaic_footprint = ee.FeatureCollection([
        ee.Feature(tile_collections[n].geometry())
        for n in order if n in tile_collections
    ]).geometry().dissolve()

    Airbusmosaic = ee.ImageCollection.fromImages(ordered_images).mosaic()
    proj = tile_collections['E'].first().select('b1').projection()
    Airbusmosaic = Airbusmosaic.setDefaultProjection(proj)

    print('Mosaic processing complete!')

In [ ]:
# =============================================================================
# VISUALIZATION: Mosaic Footprint Preview
# =============================================================================
if not PHASE_1_COMPLETE:
  import geemap
  # Create map
  Map = geemap.Map()
  # Add mosaic footprint
  Map.addLayer(
      mosaic_footprint.bounds(),
      {'color': 'red'},
      'Export Region (bounds)'
  )
  Map.addLayer(
      ee.FeatureCollection([ee.Feature(mosaic_footprint)]).style(
          fillColor='FF000020', color='FF0000', width=2
      ),
      {},
      'Mosaic Footprint (dissolved)'
  )
  # Add individual tile footprints for reference
  for name in tile_collections:
      geom = tile_collections[name].geometry()
      Map.addLayer(
          ee.FeatureCollection([ee.Feature(geom)]).style(
              fillColor='00000000', color='0000FF', width=1
          ),
          {},
          f'Tile: {name}',
          shown=False  # Hidden by default
      )
  # Center on footprint
  Map.centerObject(mosaic_footprint, zoom=9)
  Map.addLayerControl()
  print('Red = Export bounds | Blue = Individual tiles | Green = Cloud | Yellow = Management')
  Map

In [ ]:
# =============================================================================
# 9) Adding indices, textures and clip
# =============================================================================
# Daisy's ORIGINAL WORKING METHOD - simple clip()
if not PHASE_1_COMPLETE:
  Airbusmosaic = Airbusmosaic.clip(Cloud)
  Airbusmosaic = add_indices(Airbusmosaic)
  Airbusmosaic = add_texture(Airbusmosaic)
  GrasslandArea = Airbusmosaic.clip(Management).toFloat()
  print('Indices textures and clipping complete!')

In [ ]:
# =============================================================================
# 10) Export
# =============================================================================
# CONFIGURATION
# Set to None for single export, or size in meters for grid (e.g., 20000 = 20km tiles)
CELL_SIZE = None  # None = single export, 20000 = ~4 tiles, 10000 = ~15 tiles
print('Cell size set to ', f'{CELL_SIZE}')

if not PHASE_1_COMPLETE:

    print('Starting Export...')

    if CELL_SIZE is None:
        # SINGLE EXPORT
        print('Mode: Single export')
        task = ee.batch.Export.image.toAsset(
            image=GrasslandArea,
            description='GrasslandArea_Processed',
            assetId=PROCESSED_ASSET_PATH,
            region=Management.geometry().bounds(),
            scale=0.3,
            crs='EPSG:32645',
            maxPixels=1e13,
            pyramidingPolicy={'.default': 'mean'}
        )
        task.start()
        print(f'Export started: {task.id}')

    else:
        # GRID EXPORT
        print(f'Mode: Grid export (cell size = {CELL_SIZE}m)')

        grid = Management.geometry().coveringGrid('EPSG:32645', CELL_SIZE)
        grid_list = grid.toList(grid.size())
        num_tiles = grid.size().getInfo()

        print(f'Created {num_tiles} grid cells')

        for i in range(num_tiles):
            cell_geom = ee.Feature(grid_list.get(i)).geometry()

            task = ee.batch.Export.image.toAsset(
                image=GrasslandArea.clip(cell_geom),
                description=f'Grassland_Tile_{i}',
                assetId=f'{PROCESSED_ASSET_PATH}_Tile_{i}',
                region=cell_geom,
                scale=0.3,
                crs='EPSG:32645',
                maxPixels=1e13,
                pyramidingPolicy={'.default': 'mean'}
            )
            task.start()

        print(f'{num_tiles} tasks submitted!')

Cell size set to  None


---
# Phase 2: Classification Pipeline
**Run this section after `PHASE_1_COMPLETE = True`**

In [ ]:
# =============================================================================
# 1) Load Processed Data
# =============================================================================
if PHASE_1_COMPLETE:
    print('Loading processed data for Phase 2...')

    # Check if we used single export or grid export based on CELL_SIZE
    if CELL_SIZE is None:
        # SINGLE ASSET MODE
        print(f'Loading single asset: {PROCESSED_ASSET_PATH}')
        try:
            GrasslandArea = ee.Image(PROCESSED_ASSET_PATH)
            GrasslandArea = GrasslandArea.setDefaultProjection('EPSG:32645', scale=0.3)
            print('✓ Single asset loaded successfully!')
        except Exception as e:
            raise Exception(f"Failed to load asset: {e}\nCheck if export completed in Tasks tab.")

    else:
        # GRID TILES MODE
        print(f'Scanning for grid tiles (CELL_SIZE={CELL_SIZE})...')
        parent_folder = '/'.join(PROCESSED_ASSET_PATH.split('/')[:-1])
        target_base = PROCESSED_ASSET_PATH.split('/')[-1]

        try:
            assets_list = ee.data.listAssets({'parent': parent_folder})['assets']

            # Find tiles matching our export pattern
            tile_ids = [
                asset['name'] for asset in assets_list
                if target_base in asset['name'] and '_Tile_' in asset['name']
            ]

            if not tile_ids:
                raise Exception(f"No tiles found matching '{target_base}_Tile_*' in {parent_folder}")

            print(f'Found {len(tile_ids)} tiles. Building mosaic...')

            tile_collection = ee.ImageCollection(tile_ids)
            GrasslandArea = tile_collection.mosaic().setDefaultProjection('EPSG:32645', scale=0.3)

            print('✓ Tile mosaic loaded successfully!')

        except Exception as e:
            raise Exception(f"Error loading tiles: {e}\nCheck if exports completed in Tasks tab.")

    # Load vectors for Phase 2
    Management = ee.FeatureCollection('projects/daisyoneill/assets/ManagementData')
    Invasive = ee.FeatureCollection('projects/daisyoneill/assets/InvasiveClasses')

    print('Phase 2 Ready!')
else:
    # Phase 1 - GrasslandArea already in memory
    Invasive = ee.FeatureCollection('projects/daisyoneill/assets/InvasiveClasses')
    print('Phase 1 active - GrasslandArea already in memory')

Loading processed data for Phase 2...
Loading single asset: projects/quantum-bonus-434714-t2/assets/GrasslandAreaProcessed
✓ Single asset loaded successfully!
Phase 2 Ready!


In [ ]:
# =========================================================
# 2) DEEP VALIDATION OF EXPORTED ASSETS
# =========================================================try:
if 'GrasslandArea' in globals():
    print("\n--- GENERATING INSPECTION MAP ---")
    Map = geemap.Map()

    # Viz params for Airbus (12-bit data usually 0-4095)
    viz_rgb = {'bands': ['b1', 'b2', 'b3'], 'min': 0, 'max': 4095}

    Map.centerObject(Management, 14)
    Map.addLayer(GrasslandArea, viz_rgb, 'Processed Mosaic (RGB)')
    Map.addLayer(Management, {'color': 'yellow'}, 'Management Area')

    # If we have points loaded, show them too
    if 'TrainingPoints' in globals():
          Map.addLayer(TrainingPoints, {'color': 'red'}, 'Training Points')

    display(Map)


--- GENERATING INSPECTION MAP ---


Map(center=[27.544437489470923, 84.40485696101656], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# =============================================================================
# 3) ROBUST DATA EXTRACTION & DIAGNOSTIC
# =============================================================================
print('--- COMPREHENSIVE DATA EXTRACTION DIAGNOSTIC ---')
# 1. Setup projection & Points
projection = ee.Projection('EPSG:32645')
GrasslandArea_Safe = GrasslandArea.setDefaultProjection(projection)
# Input points
AllPoints = Invasive.map(lambda f: f.set('class', f.get('Mikania')))
total_input = AllPoints.size().getInfo()
# 2. Force Extraction of ALL Points (including masked ones)
# We unmask the image first to ensure sampleRegions sees everything.
# We add a validity band ('valid_mask') to track which were originally masked.
valid_mask = GrasslandArea_Safe.mask().select(0).rename('is_valid')
UnmaskedImage = GrasslandArea_Safe.unmask(-9999).addBands(valid_mask)
print(f"Sampling {total_input} points against unmasked imagery...")
# Sample everything
# geometries=True is needed to debug locations if needed
AllSamples = UnmaskedImage.sampleRegions(
    collection=AllPoints,
    properties=['class'],
    scale=0.3,
    geometries=True,
    tileScale=16
)
# 3. Categorize the Results
# Category A: Valid Data (is_valid == 1 AND not -9999)
ValidData = AllSamples.filter(ee.Filter.eq('is_valid', 1)) \
                      .filter(ee.Filter.neq('b1', -9999)) # Check one band
# Category B: Masked Pixels (is_valid == 0)
# These fell on pixels masked out by Cloud or Management
MaskedData = AllSamples.filter(ee.Filter.eq('is_valid', 0))
# Category C: No Data (is_valid == 1 but values are -9999)
# These fell inside bounds but had no data (e.g. gaps between tiles)
NoData = AllSamples.filter(ee.Filter.eq('is_valid', 1)) \
                   .filter(ee.Filter.eq('b1', -9999))
# Counts
count_total_returned = AllSamples.size().getInfo()
count_valid = ValidData.size().getInfo()
count_masked = MaskedData.size().getInfo()
count_nodata = NoData.size().getInfo()
# Calculate silently dropped (outside footprint)
count_dropped_spatially = total_input - count_total_returned
print("\n" + "="*40)
print(f"INPUT POINTS:             {total_input}")
print("="*40)
print(f"✅ VALID SAMPLES:          {count_valid}")
print(f"❌ MASKED PIXELS:          {count_masked} (Hit Cloud/Mgmt mask)")
print(f"❌ NO DATA (Gaps):         {count_nodata} (Inside bounds, no value)")
print(f"❌ OUTSIDE FOOTPRINT:      {count_dropped_spatially} (Completely outside)")
print("-" * 40)
print(f"TOTAL ACCOUNTED FOR:      {count_valid + count_masked + count_nodata + count_dropped_spatially}")
print("="*40)
# Check sanity
if count_valid == 0:
    raise RuntimeError("CRITICAL: No valid samples available for training!")
print(f"\nProceeding with {count_valid} clean samples...")

--- COMPREHENSIVE DATA EXTRACTION DIAGNOSTIC ---
Sampling 252 points against unmasked imagery...

INPUT POINTS:             252
✅ VALID SAMPLES:          138
❌ MASKED PIXELS:          114 (Hit Cloud/Mgmt mask)
❌ NO DATA (Gaps):         0 (Inside bounds, no value)
❌ OUTSIDE FOOTPRINT:      0 (Completely outside)
----------------------------------------
TOTAL ACCOUNTED FOR:      252

Proceeding with 138 clean samples...


In [ ]:
# =============================================================================
# 4) 70/30 RANDOM CROSS-VALIDATION
# =============================================================================
print('--- RUNNING 70/30 RANDOM CROSS-VALIDATION ---')
# 1. Prepare Valid Data (remove helper bands)
# We remove 'is_valid' and original class if needed, keeping just bands + class
CleanData = ValidData.select(GrasslandArea.bandNames().add('class'))
# 2. Add Random Column for Splitting
# This assigns a random float 0-1 to every feature
CleanData = CleanData.randomColumn('random', seed=42)  # Seed for reproducibility
# 3. Split Data
SPLIT = 0.7
TrainingSet = CleanData.filter(ee.Filter.lt('random', SPLIT))
ValidationSet = CleanData.filter(ee.Filter.gte('random', SPLIT))
train_n = TrainingSet.size().getInfo()
val_n = ValidationSet.size().getInfo()
print(f"Training Samples (70%):   {train_n}")
print(f"Validation Samples (30%): {val_n}")
if train_n == 0 or val_n == 0:
    raise RuntimeError("Split resulted in empty set! Check sample counts.")

--- RUNNING 70/30 RANDOM CROSS-VALIDATION ---
Training Samples (70%):   86
Validation Samples (30%): 52


In [ ]:
# =============================================================================
# 5) OPTIMIZED RF CLASSIFICATION (TUNING + THRESHOLD OPTIMIZATION + IMPORTANCE)
# =============================================================================
print('--- HYPERPARAMETER TUNING & OPTIMIZATION ---')
# 1. Hyperparameter Grid Search (Manual Implementation in GEE)
# GEE doesn't have auto-tuning, so we loop through key params
trees_list = [100, 300, 500]
min_leaf_list = [1, 3, 5]
variables_per_split_list = [None, 5, 10]  # None = sqrt(features) (default)
best_accuracy = 0.0
best_params = {}
best_model = None
print("Tuning hyperparameters (grid search)...")
# Note: Iterating in Python client, executing on server
# We use the VALIDATION set to pick the best params (standard practice)
for t in trees_list:
    for ml in min_leaf_list:
        for v in variables_per_split_list:

            # Train candidate
            rf_candidate = ee.Classifier.smileRandomForest(
                numberOfTrees=t,
                minLeafPopulation=ml,
                variablesPerSplit=v
            ).train(features=TrainingSet, classProperty='class', inputProperties=GrasslandArea.bandNames())

            # Validate
            # (Using OUTPUT_MODE='CLASSIFICATION' for tuning accuracy)
            val_matrix = ValidationSet.classify(rf_candidate).errorMatrix('class', 'classification')
            acc = val_matrix.accuracy().getInfo()

            if acc > best_accuracy:
                best_accuracy = acc
                best_params = {'trees': t, 'minLeaf': ml, 'varsPerSplit': v}
                best_model = rf_candidate
print(f"✓ Best Hyperparameters Found: {best_params}")
print(f"  Validation Accuracy: {best_accuracy:.4f}")
# 2. Variable Importance (From Best Model)
print("\n--- VARIABLE IMPORTANCE ---")
importance_dict = best_model.explain().get('importance').getInfo()
# Sort and display top 10
sorted_importance = sorted(importance_dict.items(), key=lambda item: item[1], reverse=True)
print("Top 10 Most Important Features:")
for i, (name, score) in enumerate(sorted_importance):
    print(f"  {i+1}. {name:<15} : {score:.2f}")
# 3. Probability Threshold Optimization (Maximizing F1)
print("\n--- OPTIMIZING DECISION THRESHOLD ---")
# Get Probability outputs for Validation Set
# We need to re-train in PROBABILITY mode with best params
rf_prob = ee.Classifier.smileRandomForest(
    numberOfTrees=best_params['trees'],
    minLeafPopulation=best_params['minLeaf'],
    variablesPerSplit=best_params['varsPerSplit']
).setOutputMode('PROBABILITY').train(features=TrainingSet, classProperty='class', inputProperties=GrasslandArea.bandNames())
# Classify validation points to get probabilities
val_probs = ValidationSet.classify(rf_prob)
# Test thresholds from 0.1 to 0.9
thresholds = [i/20.0 for i in range(2, 19)] # 0.1, 0.15 ... 0.9
best_f1 = 0.0
optimal_thresh = 0.5
print(f"{'Threshold':<10} | {'F1 Score':<10} | {'Kappa':<10}")
print("-" * 35)
for thresh in thresholds:
    # Manual confusion matrix calculation
    # True Positive: Class=1 AND Prob > Thresh
    tp = val_probs.filter(ee.Filter.And(ee.Filter.eq('class', 1), ee.Filter.gt('classification', thresh))).size().getInfo()
    # False Positive: Class=0 AND Prob > Thresh
    fp = val_probs.filter(ee.Filter.And(ee.Filter.eq('class', 0), ee.Filter.gt('classification', thresh))).size().getInfo()
    # False Negative: Class=1 AND Prob <= Thresh
    fn = val_probs.filter(ee.Filter.And(ee.Filter.eq('class', 1), ee.Filter.lte('classification', thresh))).size().getInfo()

    # F1 Calculation
    if (2*tp + fp + fn) > 0:
        f1 = (2 * tp) / (2 * tp + fp + fn)
    else:
        f1 = 0.0

    if f1 > best_f1:
        best_f1 = f1
        optimal_thresh = thresh

    print(f"{thresh:<10.2f} | {f1:<10.4f} | {'-'}")
print("-" * 35)
print(f"✓ Optimal Threshold: {optimal_thresh} (Max F1: {best_f1:.4f})")
# 4. Apply Final Optimized Classification
print("\n--- FINAL CLASSIFICATION ---")
# Apply optimal threshold to entire image
# 1. Get Probability Map
prob_map = GrasslandArea_Safe.classify(rf_prob)
# 2. Apply Threshold
final_classified = prob_map.gt(optimal_thresh).rename('classification')
print("Classification complete using optimized parameters and threshold.")

--- HYPERPARAMETER TUNING & OPTIMIZATION ---
Tuning hyperparameters (grid search)...
✓ Best Hyperparameters Found: {'trees': 100, 'minLeaf': 1, 'varsPerSplit': 10}
  Validation Accuracy: 0.8462

--- VARIABLE IMPORTANCE ---
Top 10 Most Important Features:
  1. ev_ent          : 8.26
  2. ev_var          : 8.18
  3. b5_var          : 7.53
  4. wi              : 6.71
  5. ev              : 6.01
  6. b1_var          : 5.81
  7. b4_var          : 5.45
  8. sa              : 4.78
  9. b6_var          : 4.69
  10. b6_contrast     : 4.51
  11. b2_var          : 4.37
  12. b1_contrast     : 4.31
  13. ev_contrast     : 4.24
  14. b2              : 4.03
  15. nd              : 3.91
  16. b5_ent          : 3.90
  17. b4              : 3.88
  18. b6_ent          : 3.85
  19. b1_ent          : 3.62
  20. b6              : 3.60
  21. b3              : 3.46
  22. b2_contrast     : 3.44
  23. b3_ent          : 3.36
  24. b5              : 3.29
  25. b2_ent          : 3.23
  26. b3_contrast     : 3.13


In [ ]:
# =============================================================================
# 6) TRAIN FINAL MODEL (OPTIMIZED)
# =============================================================================
print('Training Final Optimized Random Forest...')
print('- Trees: 100')
print('- Min Leaf: 1')
print('- Variables per Split: 10')
print('- Decision Threshold: 0.55')
# 1. Train Probability Model with Tuned Hyperparameters
rf_optimized = ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    minLeafPopulation=1,
    variablesPerSplit=10
).setOutputMode('PROBABILITY').train(
    features=TrainingSet,
    classProperty='class',
    inputProperties=GrasslandArea.bandNames()
)
# 2. Classify Image (Probability -> Threshold)
prob_map = GrasslandArea_Safe.classify(rf_optimized)
classified = prob_map.gt(0.55).rename('classification') # Applying 0.55 threshold
print('Classification complete!')

Training Final Optimized Random Forest...
- Trees: 100
- Min Leaf: 1
- Variables per Split: 10
- Decision Threshold: 0.55
Classification complete!


In [ ]:
# =============================================================================
# 7) FINAL ACCURACY ASSESSMENT (DETAILED BY CLASS)
# =============================================================================
print('Final Accuracy Assessment (Optimized Threshold 0.55)')
print('=' * 60)
# 1. Get Predictions for Validation Set
val_probs = ValidationSet.classify(rf_optimized)
val_hard = val_probs.map(lambda f: f.set('classification',
    ee.Number(f.get('classification')).gt(0.55)))
# 2. Calculate Confusion Matrix
cm = val_hard.errorMatrix('class', 'classification')
# 3. Extract and Sanitize Info
info = cm.getInfo()
accuracy = cm.accuracy().getInfo()
kappa = cm.kappa().getInfo()
f1 = cm.fscore().getInfo()
# UA/PA come back as 1x2 or 2x1 matrices: [[class0, class1]]
ua = cm.consumersAccuracy().getInfo()[0]
pa = cm.producersAccuracy().project([0]).getInfo() # Flatten the 2x1
print(f"Overall Accuracy:  {accuracy:.4f}")
print(f"Kappa Coefficient: {kappa:.4f}")
print("-" * 30)
# Mikania Presence (Class 1)
print(f"MIKANIA PRESENCE (Class 1):")
print(f"  - F1 Score:           {f1[1]:.4f}")
print(f"  - Producer's Accuracy (Recall):    {pa[1]:.4f}  (How many actual Mikania patches were found?)")
print(f"  - User's Accuracy (Precision):     {ua[1]:.4f}  (Of predicted Mikania patches, how many were correct?)")
print("-" * 30)
# Mikania Absence (Class 0)
print(f"MIKANIA ABSENCE (Class 0):")
print(f"  - F1 Score:           {f1[0]:.4f}")
print(f"  - Producer's Accuracy (Specificity): {pa[0]:.4f}")
print(f"  - User's Accuracy:                 {ua[0]:.4f}")
print("-" * 30)
print("Confusion Matrix (Actual Rows, Predicted Cols):")
print(f"           Pred 0   Pred 1")
print(f"Actual 0:  {info[0][0]:<8} {info[0][1]}")
print(f"Actual 1:  {info[1][0]:<8} {info[1][1]}")
print('=' * 60)

Final Accuracy Assessment (Optimized Threshold 0.55)
Overall Accuracy:  0.8269
Kappa Coefficient: 0.6476
------------------------------
MIKANIA PRESENCE (Class 1):
  - F1 Score:           0.8475
  - Producer's Accuracy (Recall):    0.8621  (How many actual Mikania patches were found?)
  - User's Accuracy (Precision):     0.8333  (Of predicted Mikania patches, how many were correct?)
------------------------------
MIKANIA ABSENCE (Class 0):
  - F1 Score:           0.8000
  - Producer's Accuracy (Specificity): 0.7826
  - User's Accuracy:                 0.8182
------------------------------
Confusion Matrix (Actual Rows, Predicted Cols):
           Pred 0   Pred 1
Actual 0:  18       5
Actual 1:  4        25


In [ ]:
# =============================================================================
# 8) BLOCK A: SPATIAL CV LOOP (TRAINING & STORING)
# =============================================================================
print('--- RUNNING SPATIALLY EXPLICIT CROSS-VALIDATION (OPTIMIZED) ---')
# 1. Clustering
n_folds = 5
print(f"Generating {n_folds} spatial folds...")
SpatialData = CleanData.map(lambda f: f.set('lat', f.geometry().coordinates().get(1)) \
                                       .set('lon', f.geometry().coordinates().get(0)))
clusterer = ee.Clusterer.wekaKMeans(n_folds).train(SpatialData, inputProperties=['lat', 'lon'])
SpatialData = SpatialData.cluster(clusterer, 'fold_id')
fold_distribution = SpatialData.aggregate_histogram('fold_id').getInfo()
print(f"Sample distribution per fold: {fold_distribution}")
# 2. Run Loop & STORE MODELS
accuracies = []
kappas = []
matrices = []
TRAINED_MODELS = []  # <--- Storing models here
print("\nStarting Cross-Validation Loop:")
print("-" * 60)
print(f"{'Fold':<5} | {'Train N':<10} | {'Test N':<10} | {'Accuracy':<10} | {'Kappa':<10}")
print("-" * 60)
for i in range(n_folds):
    test_fold = SpatialData.filter(ee.Filter.eq('fold_id', i))
    train_fold = SpatialData.filter(ee.Filter.neq('fold_id', i))

    # Train (Optimized + Probability Mode)
    rf = ee.Classifier.smileRandomForest(
        numberOfTrees=100,
        minLeafPopulation=1,
        variablesPerSplit=10
    ).setOutputMode('PROBABILITY').train(
        features=train_fold,
        classProperty='class',
        inputProperties=GrasslandArea.bandNames()
    )

    TRAINED_MODELS.append(rf)  # Store robust GEE object

    # Validation (Threshold 0.55)
    probs = test_fold.classify(rf)
    validated = probs.map(lambda f: f.set('classification', ee.Number(f.get('classification')).gt(0.55)))

    cm = validated.errorMatrix('class', 'classification')
    acc = cm.accuracy().getInfo() or 0.0
    k = cm.kappa().getInfo() or 0.0

    accuracies.append(acc)
    kappas.append(k)
    matrices.append(cm.getInfo())

    print(f"{i:<5} | {train_fold.size().getInfo():<10} | {test_fold.size().getInfo():<10} | {acc:.4f}     | {k:.4f}")
# 3. Summary
mean_acc = sum(accuracies) / n_folds
mean_kappa = sum(kappas) / n_folds
print("-" * 60)
print("SPATIAL CV SUMMARY")
print(f"Mean Accuracy: {mean_acc:.4f}")
print(f"Mean Kappa:    {mean_kappa:.4f}")
print("=" * 60)

--- RUNNING SPATIALLY EXPLICIT CROSS-VALIDATION (OPTIMIZED) ---
Generating 5 spatial folds...
Sample distribution per fold: {'0': 9, '1': 85, '2': 11, '3': 7, '4': 26}

Starting Cross-Validation Loop:
------------------------------------------------------------
Fold  | Train N    | Test N     | Accuracy   | Kappa     
------------------------------------------------------------
0     | 129        | 9          | 0.4444     | -0.0976
1     | 53         | 85         | 0.7529     | 0.4834
2     | 127        | 11         | 0.8182     | 0.6207
3     | 131        | 7          | 0.8571     | 0.0000
4     | 112        | 26         | 0.9231     | 0.6232
------------------------------------------------------------
SPATIAL CV SUMMARY
Mean Accuracy: 0.7592
Mean Kappa:    0.3259


In [ ]:
# =============================================================================
# 9) BLOCK B: ENSEMBLE UNCERTAINTY (REUSING MODELS)
# =============================================================================
print('--- GENERATING SPATIAL ENSEMBLE UNCERTAINTY ---')
print(f"Applying {len(TRAINED_MODELS)} pre-trained spatial models to image...")
probability_images = []
for i, model in enumerate(TRAINED_MODELS):
    # Just classify the image with the stored model
    # It is already in PROBABILITY mode from Block A
    prob_map = GrasslandArea_Safe.classify(model).rename(f'prob_fold_{i}')
    probability_images.append(prob_map)
# Stack
ensemble_stack = ee.Image.cat(probability_images)
# 1. Mean Probability
mean_prob = ensemble_stack.reduce(ee.Reducer.mean()).rename('mean_probability')
# 2. Standard Deviation (Uncertainty)
std_dev = ensemble_stack.reduce(ee.Reducer.stdDev()).rename('uncertainty_std')
# Visualize
import geemap
Map = geemap.Map()
valid_mask = GrasslandArea.select('b1').mask()
# Uncertainty
Map.addLayer(std_dev.updateMask(valid_mask),
             {'min': 0, 'max': 0.3, 'palette': ['black', 'blue', 'yellow', 'red']},
             'Spatial Uncertainty (StdDev)', True)
# Consensus Probability
Map.addLayer(mean_prob.updateMask(valid_mask),
             {'min': 0, 'max': 1, 'palette': ['white', 'green']},
             'Consensus Probability', False)
print("Spatial Uncertainty Map Generated.")
Map

--- GENERATING SPATIAL ENSEMBLE UNCERTAINTY ---
Applying 5 pre-trained spatial models to image...
Spatial Uncertainty Map Generated.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# =============================================================================
# EXPORT: BINARY CLASSIFICATION ASSET (0.3m) - SELF CONTAINED
# =============================================================================
print("--- EXPORTING BINARY CLASSIFICATION TO ASSET ---")
# 1. GEOMETRY CLEANING (Self-contained)
def clean_geometry(f):
    g = f.geometry()
    geoms = g.geometries().map(lambda geom:
        ee.Algorithms.If(ee.Geometry(geom).type().compareTo("Polygon"), None, geom), True)
    return f.setGeometry(ee.Geometry.MultiPolygon(geoms))
if 'Management' not in dir():
    Management = ee.FeatureCollection("projects/daisyoneill/assets/ManagementData")
fc_raw = Management.map(lambda f: f.set("geo_type", f.geometry().type()))
ManagementClean = fc_raw.filter(ee.Filter.neq("geo_type", "GeometryCollection")) \
    .merge(fc_raw.filter(ee.Filter.eq("geo_type", "GeometryCollection")).map(clean_geometry))
print("✓ Management geometries cleaned.")
# 2. Prepare the image (Byte format for 0/1 binary = smallest file)
ExportBinary = classified.rename('mikania_binary').byte()
# 3. Define Region (use bounds for efficiency)
export_region = ManagementClean.geometry().bounds()
# 4. Export to Asset
ASSET_ID = 'projects/quantum-bonus-434714-t2/assets/Mikania_Binary_03m'
task = ee.batch.Export.image.toAsset(
    image=ExportBinary,
    description='Mikania_Binary_03m_Asset',
    assetId=ASSET_ID,
    region=export_region,
    scale=0.3,
    crs='EPSG:32645',
    maxPixels=1e13,
    pyramidingPolicy={'mikania_binary': 'mode'}
)
task.start()
print("-" * 40)
print(f"EXPORT INITIATED")
print(f"Task ID:    {task.id}")
print(f"Asset ID:   {ASSET_ID}")
print(f"Scale:      0.3m")
print("-" * 40)

--- EXPORTING BINARY CLASSIFICATION TO ASSET ---
✓ Management geometries cleaned.
----------------------------------------
EXPORT INITIATED
Task ID:    5C6RA4275YQRO4KJTVADR6LC
Asset ID:   projects/quantum-bonus-434714-t2/assets/Mikania_Binary_03m
Scale:      0.3m
----------------------------------------


In [ ]:
# =============================================================================
# BATCHED BOOTSTRAP (EXACT TRANSLATION OF WORKING JS CODE)
# =============================================================================
print("--- BATCHED BOOTSTRAP (50 seeds per task) ---")
# 1. SETUP (Same as JS)
if 'Management' not in dir():
    Management = ee.FeatureCollection("projects/daisyoneill/assets/ManagementData")
# Geometry cleaning
fc = Management.map(lambda f: f.set('geo_type', f.geometry().type()))
gcs = fc.filter(ee.Filter.eq('geo_type', 'GeometryCollection'))
nonGcs = fc.filter(ee.Filter.neq('geo_type', 'GeometryCollection'))
def clean_gc(feature):
    filteredGeoms = feature.geometry().geometries().map(
        lambda geometry: ee.Algorithms.If(
            ee.Geometry(geometry).type().compareTo('Polygon'), None, geometry
        ), True)
    return feature.setGeometry(ee.Geometry.MultiPolygon(filteredGeoms))
fc_clean = nonGcs.merge(gcs.map(clean_gc))
# Rasterize properties
def rasterizeProperty(prop):
    distinctNames = fc_clean.aggregate_array(prop).distinct()
    distinctCodes = ee.List.sequence(1, distinctNames.size())
    lookup = ee.Dictionary.fromLists(distinctNames, distinctCodes)
    def addCode(f):
        return f.set('Code', lookup.get(f.get(prop)))
    return ee.Image().paint(fc.map(addCode), 'Code')
categoricalRaster = ee.ImageCollection([
    rasterizeProperty('Location'),
    rasterizeProperty('Mgmt2016'),
    rasterizeProperty('Mgmt2022')
]).toBands().rename(['Location', 'Mgmt2016', 'Mgmt2022'])
numericalRaster = ee.Image().paint(fc_clean, 'fid').rename('fid')
# Load classified asset
im = ee.Image('projects/quantum-bonus-434714-t2/assets/Mikania_Binary_03m').rename('classification')
# Combined image (matches your JS 'combinedIm')
combinedIm = im.addBands(categoricalRaster).addBands(numericalRaster)
# 2. SAMPLE FUNCTION (Exact translation of your JS samplePixels)
def samplePixels(seed):
    seed = ee.Number(seed)
    samples = combinedIm.sample(
        region=fc_clean,
        numPixels=10000,
        seed=seed,
        dropNulls=True
    )
    return samples.map(lambda feat: feat.set('randomSeed', seed))
# 3. BATCHED EXPORT LOOP (50 seeds per task, 20 tasks total)
print("Submitting 20 export tasks...")
for batch_start in range(1, 1001, 50):
    batch_end = batch_start + 49

    seeds = ee.List.sequence(batch_start, batch_end)
    samples = ee.FeatureCollection(seeds.map(samplePixels)).flatten()

    desc = f'samplePoints_{batch_start}_to_{batch_end}'

    task = ee.batch.Export.table.toDrive(
        collection=samples,
        description=desc,
        folder='Mikania_Bootstrap_Batches',
        fileFormat='CSV'
    )
    task.start()
    print(f"  Submitted: {desc}")
print("-" * 40)
print("20 tasks submitted (50 seeds × 10k points = 500k per task)")
print("-" * 40)

--- BATCHED BOOTSTRAP (50 seeds per task) ---
Submitting 20 export tasks...
  Submitted: samplePoints_1_to_50
  Submitted: samplePoints_51_to_100
  Submitted: samplePoints_101_to_150
  Submitted: samplePoints_151_to_200
  Submitted: samplePoints_201_to_250
  Submitted: samplePoints_251_to_300
  Submitted: samplePoints_301_to_350
  Submitted: samplePoints_351_to_400
  Submitted: samplePoints_401_to_450
  Submitted: samplePoints_451_to_500
  Submitted: samplePoints_501_to_550
  Submitted: samplePoints_551_to_600
  Submitted: samplePoints_601_to_650
  Submitted: samplePoints_651_to_700
  Submitted: samplePoints_701_to_750
  Submitted: samplePoints_751_to_800
  Submitted: samplePoints_801_to_850
  Submitted: samplePoints_851_to_900
  Submitted: samplePoints_901_to_950
  Submitted: samplePoints_951_to_1000
----------------------------------------
20 tasks submitted (50 seeds × 10k points = 500k per task)
----------------------------------------


In [ ]:
# =============================================================================
# 11) EXPORT: OPTIMIZED RESULTS (10M SCALE) - 3 BANDS
# =============================================================================
print("Preparing final results for export...")
# 1. Combine into a 3-band image
# Band 1: Mean Probability (Ensemble)
# Band 2: Uncertainty (StdDev across spatial folds)
# Band 3: Binary Classification (Optimized Threshold)
ExportImage = mean_prob.rename('mikania_prob_mean') \
    .addBands(std_dev.rename('mikania_uncertainty_std')) \
    .addBands(classified.rename('mikania_binary_optimized')) \
    .toFloat()
# 2. Constrain to Management Area
# Create single-band mask from the drawn Management layer
mgmt_mask = ManagementClean.draw(color="000000", pointRadius=1).mask().select(0)
ExportImage = ExportImage.updateMask(mgmt_mask)
# 3. Define Region
export_region = ManagementClean.geometry().bounds()
# 4. Start Export Task
task_description = 'Mikania_Results_10m_3Bands'
task = ee.batch.Export.image.toDrive(
    image=ExportImage,
    description=task_description,
    folder='Mikania_Final_Maps',
    fileNamePrefix=task_description,
    region=export_region,
    scale=10,
    crs='EPSG:32645',
    maxPixels=1e10
)
task.start()
print("-" * 30)
print(f"EXPORT INITIATED")
print(f"Task ID:     {task.id}")
print(f"Scale:       10m")
print(f"Bands:       [mikania_prob_mean, mikania_uncertainty_std, mikania_binary_optimized]")
print(f"Region:      Management Area Bounds")
print("-" * 30)
print("Check the 'Tasks' tab in the GEE Console to monitor progress.")

Preparing final results for export...
------------------------------
EXPORT INITIATED
Task ID:     BPPNJD676RJJQFAYG7NZIGBG
Scale:       10m
Bands:       [mikania_prob_mean, mikania_uncertainty_std, mikania_binary_optimized]
Region:      Management Area Bounds
------------------------------
Check the 'Tasks' tab in the GEE Console to monitor progress.
